# 体验HTML的标签

In [2]:
from IPython.core.display import HTML

In [3]:
HTML('<h1>好好学习</h1>')

In [4]:
%%HTML
<h1>好好学习</h1>

In [3]:
%%html
<a href='https://www.nowcoder.com/tutorial/10005/92bd2f50f48741fcb760575baf14e025'>天天向上</a>
#href属性，页面跳转到“天天向上”

In [5]:
%%html
<p>好好学习天天向上一定会有收获</p>

In [ ]:
%%html
<img src='https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fimg.2qqtouxiang.com%2Fpic%2FTX9734_05.jpg&refer=http%3A%2F%2Fimg.2qqtouxiang.com&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=auto?sec=1652973586&t=b3b4834a9f6a7dbafcdcca5e8c5c2591'>
#src属性

爬虫的流程就是“进-消-存”三步——访问页面(获取内容)、解析(抽取信息)、保存数据。复杂的爬虫也是如此，只是为了提高效率避免被封，多了一些并行化、调度策略等设计。

# 安装第三方库 requests/lxml

python -m pip install requests

python -m pip install lxml

## 获取网页源码（可能需要包装）

In [ ]:
import requests
r = requests.get('https://search.douban.com/book/subject_search?search_text=%E7%BC%96%E7%A8%8B&cat=1001')
r.status_code
r.headers
r.text

若无法直接get到url可以通过convert cURL to python，修改headers，伪装浏览器的方式来获取

In [65]:
import requests
from requests.structures import CaseInsensitiveDict

url = "https://search.douban.com/book/subject_search?search_text=%E7%BC%96%E7%A8%8B&cat=1001"

headers = CaseInsensitiveDict()
headers["Accept"] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
headers["Referer"] = "https://book.douban.com/"
headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"

resp = requests.get(url, headers=headers)

print(resp.status_code)

200


## 1.DOM/xpath

DOM:document object model

定位页面元素路径

In [66]:
from lxml import etree
sel = etree.HTML(resp.text)

### 用xpath获取相应内容

xpath基础语法

- 选取节点的语法有:

/ 从根节点选取

// 从所有匹配的节点选取

. 当前节点

.. 当前的父节点

nodename选取节点, @选取节点的属性

通配符: *

选取若干路径则用|分隔

text(): 获取该节点的文本内容

例子:

//img/@src: 选取所有img节点的src属性

//img/../text: 选取img节点的父节点下的text节点(所以text和img为"sibling"关系)

//*/@src: 选取任何节点的src属性

- 过滤节点的谓词语法有: (谓词放在方括号中)

[1]选取第一个元素, [last()]选取最后一个, [position<3] 选取前两个

[@lang="eng"] 选取属性lang等于"eng"的元素

### 获取单独的书名或价格（但未一一对应）

In [1]:
import requests
from requests.structures import CaseInsensitiveDict
from lxml import etree

url = "https://book.douban.com/tag/%E5%B0%8F%E8%AF%B4"

headers = CaseInsensitiveDict()
headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
resp = requests.get(url, headers=headers)
sel = etree.HTML(resp.text)

for elem in sel.xpath("//li[@class='subject-item']/div[@class='info']/h2/a/text()"):
    print(elem.strip())

for elem_p in sel.xpath("//li[@class='subject-item']/div[@class='info']/div[@class='ft']/div[@class='cart-actions']/span[@class='buy-info']/a/text()"):
    print(elem_p.strip()[4:-1])


鹿川有许多粪
生死疲劳
克拉拉与太阳
遥远的救世主
消失的13级台阶
素食者
绝叫
秋园
文城
活着
流俗地
献给阿尔吉侬的花束
夜晚的潜水艇
挽救计划
人世间
百年孤独
克莱因壶
一日三秋
索拉里斯星
海边的房间
34.90
47.00
38.40
26.10
20.00
154.70
26.30


#### 标签内嵌套标签，读取文本内容不全，itertext()

In [8]:
import requests
from requests.structures import CaseInsensitiveDict
from lxml import etree

url = 'https://book.douban.com/tag/%E6%BC%AB%E7%94%BB'
headers = CaseInsensitiveDict()
headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"

resp = requests.get(url=url, headers=headers)
dom = etree.HTML(resp.text)
# dom.xpath("//li[@class='subject-item']/div[@class='info']/h2/a/text()")  #发现标题不完整。原因a标签里嵌套span标签



['\n\n    纳闷集\n\n\n    \n\n  ',
 '\n\n    镖人\n\n\n    \n\n  ',
 '\n\n    再见绘梨\n\n\n    \n\n  ',
 '\n\n    胖虎下山\n\n\n    \n\n  ',
 '\n\n    ルックバツク\n\n\n    \n      ',
 '\n\n  ',
 '\n\n    镖人\n\n\n    \n      ',
 '\n\n  ',
 '\n\n    地狱星REMINA\n\n\n    \n\n  ',
 '\n\n    伊藤润二短篇精选集\n\n\n    \n\n  ',
 '\n\n    一个故事的99种讲法\n\n\n    \n\n  ',
 '\n\n    美丽黑暗\n\n\n    \n\n  ',
 '\n\n    间谍过家家（第一卷）\n\n\n    \n      ',
 '\n\n  ',
 '\n\n    看你一眼就会笑\n\n\n    \n\n  ',
 '\n\n    夏日時光(01)\n\n\n    \n\n  ',
 '\n\n    镖人\n\n\n    \n      ',
 '\n\n  ',
 '\n\n    镖人\n\n\n    \n      ',
 '\n\n  ',
 '\n\n    鱼（全2册）\n\n\n    \n\n  ',
 '\n\n    回答不了\n\n\n    \n\n  ',
 '\n\n    你今天真好看\n\n\n    \n\n  ',
 '\n\n    镖人\n\n\n    \n      ',
 '\n\n  ',
 '\n\n    黑睡莲\n\n\n    \n\n  ']

取文本的方式：

1.xpath末尾加上/text()

2.element元素.text/.itertext()---获取当前节点及子节点所有文本内容

In [33]:
for elem in dom.xpath("//li[@class='subject-item']/div[@class='info']/h2/a"):
    print(''.join(elem.itertext()).replace(' ','').replace('\n','')) #join可作用于迭代器

纳闷集
镖人
再见绘梨
胖虎下山
ルックバツク:LookBack
镖人:卷十一
地狱星REMINA
伊藤润二短篇精选集
一个故事的99种讲法
美丽黑暗
间谍过家家（第一卷）:1
看你一眼就会笑
夏日時光(01)
镖人:卷二
镖人:卷三
鱼（全2册）
回答不了
你今天真好看
镖人:卷四
黑睡莲


In [28]:
for pri in dom.xpath("//li[@class='subject-item']/div[@class='info']/div[@class='ft']/div[@class='cart-actions']/span[@class='buy-info']/a/text()"):
    print(float(pri.strip()[3:-1]))

39.0
44.82
45.0
17.7
34.2
39.9
22.5
44.25
29.3
18.9


### 获取书名或价格（一一对应）

element可以嵌套使用

notebook显示图片

In [70]:
from IPython.core.display import HTML
from IPython.display import display

for block in dom.xpath("//li[@class='subject-item']"):

    title = ''
    elem_title = block.xpath(".//div[@class='info']/h2/a") #是个列表
    # print(elem_title) 
    # print(elem_title[0])
    if elem_title:
        title =''.join(elem_title[0].itertext()).replace(' ','').replace('\n','')
    # print(title)

    price = -1
    elem_price =  block.xpath(".//div[@class='info']/div[@class='ft']/div[@class='cart-actions']/span[@class='buy-info']/a/text()") #是个列表
    # print(elem_price)
    if elem_price:
        price = float(elem_price[0].strip()[3:-1])
    # print(price)
    print(title,price)

    cover = ''
    elem_cover = block.xpath(".//img/@src")
    if elem_cover:
        cover = elem_cover[0]
    display(HTML(f'<img src={cover}>')) #notebook显示图片
    
    

纳闷集 -1


镖人 -1


再见绘梨 -1


胖虎下山 -1


ルックバツク:LookBack 39.0


...

鱼（全2册） 22.5


回答不了 44.25


你今天真好看 29.3


镖人:卷四 18.9


黑睡莲 -1


### 保存爬取的图片

In [5]:
import requests

url = 'https://www.natgeo.com.cn/pic/program_default.768.jpg'
r = requests.get(url)
# r.content
# with open(url.split('/')[-1],'wb') as f:
#     f.write(r.content)

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00K\x00K\x00\x00\xff\xdb\x00C\x00\x06\x04\x05\x06\x05\x04\x06\x06\x05\x06\x07\x07\x06\x08\n\x10\n\n\t\t\n\x14\x0e\x0f\x0c\x10\x17\x14\x18\x18\x17\x14\x16\x16\x1a\x1d%\x1f\x1a\x1b#\x1c\x16\x16 , #&\')*)\x19\x1f-0-(0%()(\xff\xdb\x00C\x01\x07\x07\x07\n\x08\n\x13\n\n\x13(\x1a\x16\x1a((((((((((((((((((((((((((((((((((((((((((((((((((\xff\xc0\x00\x11\x08\x01\xb0\x03\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc

### 采集多页

#### 获取下一页链接:拼接

In [73]:
#按照首字母顺序排列，除了IPython
from lxml import etree
import requests
from requests.structures import CaseInsensitiveDict
from IPython.core.display import HTML
from IPython.display import display

next_page_url = 'https://book.douban.com/tag/%E6%BC%AB%E7%94%BB'

while 1:
    headers = CaseInsensitiveDict()
    headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
    resp = requests.get(url=next_page_url, headers=headers)
    dom = etree.HTML(resp.text)

    for block in dom.xpath("//li[@class='subject-item']"):

        title = ''
        elem_title = block.xpath(".//div[@class='info']/h2/a") #是个列表
        # print(elem_title) 
        # print(elem_title[0])
        if elem_title:
            title =''.join(elem_title[0].itertext()).replace(' ','').replace('\n','')
        # print(title)

        price = -1
        elem_price =  block.xpath(".//div[@class='info']/div[@class='ft']/div[@class='cart-actions']/span[@class='buy-info']/a/text()") #是个列表
        # print(elem_price)
        if elem_price:
            price = float(elem_price[0].strip()[3:-1])
        # print(price)
        print(title,price)

        cover = ''
        elem_cover = block.xpath(".//img/@src")
        if elem_cover:
            cover = elem_cover[0]
        # display(HTML(f'<img src={cover}>')) #notebook显示图片

    elem_next_page = dom.xpath("//span[@class='next']/a/@herf") #/@herf 源url属性 
    if elem_next_page:
        next_page_url = "https://book.douban.com" + elem_next_page[0]
    else:
        break
    
    break #可以设置翻几页


#### url中找规律

In [ ]:
#按照首字母顺序排列，除了IPython
from lxml import etree
import requests
from requests.structures import CaseInsensitiveDict
import time
from IPython.core.display import HTML
from IPython.display import display

page = 1

while 1:
    next_page_url = f'https://book.douban.com/tag/%E6%BC%AB%E7%94%BB?start={(page-1)*20}%type=T' #若没有将中文自动化转译，需要手工转移
    headers = CaseInsensitiveDict()
    headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
    resp = requests.get(url=next_page_url, headers=headers)
    dom = etree.HTML(resp.text)

    for block in dom.xpath("//li[@class='subject-item']"):

        title = ''
        elem_title = block.xpath(".//div[@class='info']/h2/a") #是个列表
        # print(elem_title) 
        # print(elem_title[0])
        if elem_title:
            title =''.join(elem_title[0].itertext()).replace(' ','').replace('\n','')
        # print(title)

        price = -1
        elem_price =  block.xpath(".//div[@class='info']/div[@class='ft']/div[@class='cart-actions']/span[@class='buy-info']/a/text()") #是个列表
        # print(elem_price)
        if elem_price:
            s = elem_price[0].strip()
            price = float(s[s.find("版")+2:s.find("元")]) #find 查找字符串的位置，若不存在，返回-1。切片，左闭右开。
        # print(price)
        print(title,price)

        cover = ''
        elem_cover = block.xpath(".//img/@src")
        if elem_cover:
            cover = elem_cover[0]
        # display(HTML(f'<img src={cover}>')) #notebook显示图片
    page += 1
       
    time.sleep(5)

    if page > 4:
        break #可以设置翻几页，并且sleep(),不然太快了会被封

### 存储为CSV，写和读

In [ ]:
#按照首字母顺序排列，除了IPython
from lxml import etree
import requests
from requests.structures import CaseInsensitiveDict
import time
from IPython.core.display import HTML
from IPython.display import display

book_info = []
page = 1

while 1:
    next_page_url = f'https://book.douban.com/tag/%E6%BC%AB%E7%94%BB?start={(page-1)*20}%type=T' #若没有将中文自动化转译，需要手工转移
    headers = CaseInsensitiveDict()
    headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36"
    resp = requests.get(url=next_page_url, headers=headers)
    dom = etree.HTML(resp.text)

    for block in dom.xpath("//li[@class='subject-item']"):

        title = ''
        elem_title = block.xpath(".//div[@class='info']/h2/a") #是个列表
        # print(elem_title) 
        # print(elem_title[0])
        if elem_title:
            title =''.join(elem_title[0].itertext()).replace(' ','').replace('\n','')
        # print(title)

        price = -1
        elem_price =  block.xpath(".//div[@class='info']/div[@class='ft']/div[@class='cart-actions']/span[@class='buy-info']/a/text()") #是个列表
        # print(elem_price)
        if elem_price:
            s = elem_price[0].strip()
            price = float(s[s.find("版")+2:s.find("元")])
        # print(price)
        print(title,price)

        cover = ''
        elem_cover = block.xpath(".//img/@src")
        if elem_cover:
            cover = elem_cover[0]
        # display(HTML(f'<img src={cover}>')) #notebook显示图片
        book_info.append([title,cover,price])
    page += 1
       
    time.sleep(5)

    if page > 4:
        break #可以设置翻几页，并且sleep(),不然太快了会被封

In [ ]:
import csv

with open('books.csv','w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(book_info)

with open('books.csv','r') as f:
    csv_reader = csv.reader(f)
    for line in csv_reader:
        print(line)

##豆瓣提供了API可以直接用

## 2.页面动态加载Ajax

问题：dom.xpath()返回空列表

怀疑内容不是页面包含的，而是javascript取到的

右击内容-查看页面源码-发现里面没有该内容-说明使用javascript动态采集?

In [1]:
import requests
from lxml import etree

url = 'https://www.bilibili.com/v/popular/all?spm_id_from=333.851.b_7072696d61727950616765546162.3'

r = requests.get(url=url)
dom = etree.HTML(r.text)
dom.xpath("//div[@class='video-card']/div[@class='video-card__info']/p[@class='video-name']")

[]

#### AJAX 

= Asynchronous JavaScript and XML（异步的 JavaScript 和 XML）。

AJAX 不是新的编程语言，而是一种使用现有标准的新方法。

AJAX 最大的优点是在不重新加载整个页面的情况下，可以与服务器交换数据并更新部分网页内容。

AJAX 不需要任何浏览器插件，但需要用户允许JavaScript在浏览器上执行。

In [2]:
%%HTML
<img src='https://tse1-mm.cn.bing.net/th/id/R-C.7c56ffd3524aba660fb81b8b1c37e751?rik=NxP4uXi80Lceug&riu=http%3a%2f%2fimage.yanhongzhi.com%2fimage%2fjava_web%2fajax%2f15460590782610.jpg&ehk=59AXJV1%2fbNH5dV%2fm%2fx8gBe3eP%2fm%2bFCTob1SlXq8G2U8%3d&risl=&pid=ImgRaw&r=0'>

开发者工具-network-xhr

表示通过javascript远程读取数据的内容

#### JSON

JavaScript Object Notation, more commonly known by the acronym JSON, is an open data interchange format that is both human and machine-readable. Despite the name JavaScript Object Notation, JSON is independent of any programming language and is a common API output in a wide variety of applications.

JSON: JavaScript Object Notation(JavaScript 对象表示法)

JSON 是存储和交换文本信息的语法，类似 XML。

JSON 比 XML 更小、更快，更易解析。

json本身用类似字典的嵌套结构直接定义了数据的结构，要获取某项数据只需要知道该数据在嵌套结构中的位置就可以；html本身是靠嵌套的标签形成的树状结构，要获取某项数据需要知道该数据在树状结构中的位置，而xpath就是在用dom表示的这个树状结构里表示确定位置信息的“路径”。

In [1]:
import requests
from lxml import etree
url = 'https://api.bilibili.com/x/web-interface/popular?ps=20&pn=1'
resp = requests.get(url=url)
resp.status_code

200

In [2]:
#返回json格式，需要json解析
resp.text

'{"code":0,"message":"0","ttl":1,"data":{"list":[{"aid":854174470,"videos":1,"tid":47,"tname":"短片·手书·配音","copyright":1,"pic":"http://i0.hdslb.com/bfs/archive/0d7b33a530d580b9f21733d59ade480c245d5299.jpg","title":"《 爷 爷 拯 救 不 完 了 》","pubdate":1652781788,"ctime":1652781447,"desc":"(《梦幻西游》电脑版联动葫芦兄弟活动现已开启，有变身卡后是爷爷守护葫芦娃？还是葫芦娃守护爷爷？到梦幻三界和葫芦兄弟们“葫”闹蛇精！)\\n我呢，有一个朋友，啊当然这个朋友并不是我自己啊\\n是个在津都这座钢铁丛林中还梳着飞机头的飘逸男子\\n平时比较得瑟，上班时候还喜欢揩油自己有好感的同事\\n对此公司中的男同事是苦不堪言，叫苦不迭\\n他们领导平时也会劝兑他\\n说人生在世呢，多多少少要收敛一点，不能太狂放不羁\\n要么说这领导也是好领导\\n日以继夜的劝导下，我朋友展现出一幅醍醐灌顶的样子\\n领导一看很开心，说：\\n“只要你以后能改，我","state":0,"duration":983,"rights":{"bp":0,"elec":0,"download":0,"movie":0,"pay":0,"hd5":1,"no_reprint":1,"autoplay":1,"ugc_pay":0,"is_cooperation":0,"ugc_pay_preview":0,"no_background":0,"arc_pay":0,"pay_free_watch":0},"owner":{"mid":158119101,"name":"远古黑金","face":"http://i0.hdslb.com/bfs/face/2466b9fb0482ea9ea6bc3488306b5557771c5d79.jpg"},"stat":{"aid":854174470,"view":1099700,"danmaku":4819,"reply":1574,"favorite":21123,"coin":26100,"sha

In [3]:
#json()方法将响应内容转换成Python能够解析的格式，即字典，可以通过字典的方式进行访问
resp.json()

{'code': 0,
 'message': '0',
 'ttl': 1,
 'data': {'list': [{'aid': 854174470,
    'videos': 1,
    'tid': 47,
    'tname': '短片·手书·配音',
    'copyright': 1,
    'pic': 'http://i0.hdslb.com/bfs/archive/0d7b33a530d580b9f21733d59ade480c245d5299.jpg',
    'title': '《 爷 爷 拯 救 不 完 了 》',
    'pubdate': 1652781788,
    'ctime': 1652781447,
    'desc': '(《梦幻西游》电脑版联动葫芦兄弟活动现已开启，有变身卡后是爷爷守护葫芦娃？还是葫芦娃守护爷爷？到梦幻三界和葫芦兄弟们“葫”闹蛇精！)\n我呢，有一个朋友，啊当然这个朋友并不是我自己啊\n是个在津都这座钢铁丛林中还梳着飞机头的飘逸男子\n平时比较得瑟，上班时候还喜欢揩油自己有好感的同事\n对此公司中的男同事是苦不堪言，叫苦不迭\n他们领导平时也会劝兑他\n说人生在世呢，多多少少要收敛一点，不能太狂放不羁\n要么说这领导也是好领导\n日以继夜的劝导下，我朋友展现出一幅醍醐灌顶的样子\n领导一看很开心，说：\n“只要你以后能改，我',
    'state': 0,
    'duration': 983,
    'rights': {'bp': 0,
     'elec': 0,
     'download': 0,
     'movie': 0,
     'pay': 0,
     'hd5': 1,
     'no_reprint': 1,
     'autoplay': 1,
     'ugc_pay': 0,
     'is_cooperation': 0,
     'ugc_pay_preview': 0,
     'no_background': 0,
     'arc_pay': 0,
     'pay_free_watch': 0},
    'owner': {'mid': 158119101,
     'name': '远古黑金',
     

In [4]:
json_content = resp.json()
for item in json_content['data']['list']:
    print(item['title'])

《 爷 爷 拯 救 不 完 了 》
做出来之后玩了四个小时 愣是没舍得吃
我打赌，这个猫咪做出来你能玩一天
沙县小吃史上第一条广告！
大爷说这幅最好看，说这是画了两个小年轻在处对象哈哈
云南的和不是云南的都沉默了
云 南 战 车
全文都看不懂，七选五次次满分
我 是 日 本 的
【战斗首曝】逆水寒手游开启预约，武侠开放世界与战斗实机画面前瞻
《 不 醒 人 室 》
用时一个多月成功骑行穿越阿里中线，亚热乡水井房扎营休息一天
【罗翔】结核病人上学记，学生可以告学校吗？
“打倒北约 打倒帝国主义”
为什么我国不强制服兵役
30+连败赏金猎人？我能创造历史吗？
小蓝鲸  厨子探店¥367
骂骂咧咧地夸这种铁线莲
《 卷 死 自 己 》
当年唐国强老师拍的山东蓝翔的广告原来是这样拍出来的！


In [11]:
import requests
#实现翻页
page = 1
while 1:
    url = f'https://api.bilibili.com/x/web-interface/popular?ps=20&pn={page}'   
    resp = requests.get(url=url)
    json_content = resp.json()

    for item in json_content['data']['list']:
        print(item['title'])
    
    page += 1
    if page > 3:
        break

再现绝美胡旋舞！还原李白眼中的汉唐夜宴
如何偷走一架飞机
满满海佛跳墙 厨子探店¥1？13
探秘墨西哥街头美食，玉米病菌真的好吃吗？
进来让世界感受一下什么是内卷王中王！！
“短短几秒，央视记者就泪崩了，队长，快救救我”
它改编自最残酷的现实，却被痛批价值观不正确
河南农民翻看女儿课本，却发现自己牺牲，身份曝光引军区高度重视
三 年 疫 情，欲 望 逐 减
【罗翔】人身保护令也管不了张三虐待儿子？读评论#15
“你们就是这样对待学长吗”
拜登羞辱普通军人视频曝光：富人对战争才有发言权 你没有
这个视频让你明白，任天堂到底有多牛？【生意02】
《本草纲目》，加亿点点难度
国防部正告美方：生化危机制造者必须作出澄清，光躲是躲不过去的
b站网友写诗，一首比一首奇怪! !
哟呼 跟主人一起回家
多少个赞，明天射多少箭。（几个月没射箭了）
中国人不骗中国人！4男子伪装性别实施诈骗，专骗日本男网友
【漫威/高燃】爆肝3个月，只为这七分钟视听盛宴！Avengers Assemble！
游戏内容及互动治理专项！3月社区治理月报
这是碳基生物能做出的事吗
范闲啥时候复活？！《庆余年》爆笑解说第15期(大结局)
什么麻雀能受得了这三重刺激？
《 本 草 纲 目 》 ，上 点 难 度
这样子男生可以理解了吧
总统竟然向毒贩求饶，全世界的禁毒战争还有希望赢吗？【乌鸦校尉】
【原神/𝙎𝙝𝙖𝙙𝙤𝙬 𝙊𝙛 𝙏𝙝𝙚 𝙎𝙪𝙣】来自提瓦特大陆的视觉盛宴 !
恋爱初期郊游！
当我们需要合成收集一百种「阴间方块」？？！
当我突然去接跳舞的女朋友下课 反差萌的女朋友真的sha 我！
给你机会你不中用呀 - iPhonese3【值不值得买第547期】
《这只猫有点猪》
女孩无意间拍到对面小伙打电话，结果逐渐变成了一部“居家连续剧”，网友：这男人有点上头！
重庆火烧铁板！老板当面玩火，直接把我看傻了…
《崩坏：星穹铁道》漫游PV：「你」的选择
你管这叫植物大战僵尸？#4
我把自行车骑上了世界上海拔最高的公路垭口
“我好像做了个很长的超英梦....”
【极限打工人】一天打12份工，“30岁前存100万”
蓝翔技校校长女儿荣婷实名举报母亲非法转移房产
“有些人表面是笨蛋 实际上是…..”
高虐｜本草纲目来啦！
曾经火遍B站的《霜雪千年》，还有多少人记得当年的音乐区和古风圈？
你用函数写作业，我用函数 做 动 画
神

## 3.正则表达式爬取网页

In [34]:
import requests
from lxml import etree

url = 'http://www.zazhipu.com/listb6_1_1_0_0.html'
resp = requests.get(url=url)
resp.status_code

200

In [36]:
resp.encoding = 'gb2312'

In [37]:
resp.text

'\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n\t<head>\r\n\t\t<meta charset="GB2312">\r\n\t\t<title>地理 旅游 人文 杂志订阅_地理 旅游 人文_ 杂志铺:低价、便捷的杂志折扣订阅网</title>\r\n\t\t<META NAME="Keywords" CONTENT=" 地理 旅游 人文 杂志订阅,杂志订购,地理 旅游 人文_ 杂志铺,低价,便捷的杂志折扣订阅网">\r\n        <META  Name="description" CONTENT="地理 旅游 人文杂志订阅,杂志订购,提供地理 旅游 人文杂志低价优惠订阅、订购,杂志铺:低价、便捷的杂志折扣订阅网，提供上万种时尚杂志、财经杂志、旅游地理杂志、文学杂志、婴儿杂志、幼儿杂志、汽车杂志、家居杂志、专业杂志订阅、订购服务。">\r\n\t\t<link rel="stylesheet" href="newindexfile/css/swiper.min.css" />\r\n\t\t<link rel="stylesheet" href="newindexfile/css/public.css" />\r\n\t\t<link rel="stylesheet" href="newfiles/css/searchresult.css" />\r\n\t\t<script type="text/javascript" src="newindexfile/js/jquery-3.1.0.min.js" ></script>\r\n\t\t<script type="text/javascript" src="newindexfile/js/swiper.min.js" ></script>\r\n\t\t<script type="text/javascript" src="https://dl.ntalker.com/js/xn6/ntkfstat.js?siteid=kf_10182"></script>\r\n\t\t<script type="text/javascript" src="newfiles/js/searchresult.js" ></script>\r\n\t<meta http-equiv="

In [38]:
content = resp.text

In [15]:
# 通过正则表达式提取内容
import re

In [24]:
re.findall('<title>([\S\s]*?)<\/title>',content)

['地理 旅游 人文 杂志订阅_地理 旅游 人文_ 杂志铺:低价、便捷的杂志折扣订阅网']

In [26]:
title_matches = re.findall('<title>([\S\s]*?)<\/title>',content)
title_matches[0].strip()

'地理 旅游 人文 杂志订阅_地理 旅游 人文_ 杂志铺:低价、便捷的杂志折扣订阅网'

In [61]:
#列表去重不改变顺序
result = re.findall('<a href="http:\/\/www.zazhipu\.com\/\d*\.html" target="_blank">.*?<\/a>',content)
li = list(set(result))
li.sort(key=result.index)
li

In [66]:
i = 0
for item in li:
    item1 = re.sub('<a.*?img.*?>','',item).replace('</a>','')
    item2 = re.sub('<a.*?>','',item1)
    
    if item2.split():
        i += 1
        print(item2)
print(i)

KiDS环球少年地理（与美国国家地理少儿版版权合作）（1年共12期）（杂志订阅）
孤独星球（Lonely Planet Magazine国际中文版）（1年共12期）（杂志订阅）
美国国家地理（英文原版）National Geographic （1年共12期）（杂志订阅）
探索地理（1年共12期）（杂志订阅）
旅行家（1年共12期）杂志订阅
大自然探索（1年共12期）（杂志订阅）
中国三峡河流地理与水文化（杂志订阅）（1年共12期）
美国国家地理幼儿版3-6岁（英文原版）National Geographic  Little Kids（1年共6期）（杂志订阅）
美国国家地理儿童版（英文原版）National Geographic Kids（1年共10期）（杂志订阅）
中国国家地理（1年共12期）（杂志订阅）
海洋探秘（1年共12期）（杂志订阅）
天府文化（1年共12期）（杂志订阅）
自然探秘(1年共12期）（杂志订阅）
户外探险（1年共6期）（杂志订阅）
旅游世界（1年共12期）（杂志订阅）
四川画报（1年共12期）（杂志订阅）
包邮 KiDS环球少年地理（与美国国家地理少儿版版权合作）（1年共12期）（杂志订阅）
包邮 中国国家地理（1年共12期）（杂志订阅）
动物大世界 中国儿童画报（1年共12期）（杂志订阅）【杂志铺专供】
快乐科学 快乐历史地理（1年共12期）（杂志订阅）
20


试了一下用xpath获取

In [74]:
dom = etree.HTML(content)
dom_xpath = dom.xpath("//div[@class='searchresult_item_t_l']/a/text()")

In [75]:
list_dom_xpath = list(set(dom_xpath))
list_dom_xpath.sort(key=dom_xpath.index)

In [77]:
len(list_dom_xpath)

20

In [78]:
for elem in list_dom_xpath:
    print(elem)

KiDS环球少年地理（与美国国家地理少儿版版权合作）（1年共12期）（杂志订阅）
孤独星球（Lonely Planet Magazine国际中文版）（1年共12期）（杂志订阅）
美国国家地理（英文原版）National Geographic （1年共12期）（杂志订阅）
探索地理（1年共12期）（杂志订阅）
旅行家（1年共12期）杂志订阅
大自然探索（1年共12期）（杂志订阅）
中国三峡河流地理与水文化（杂志订阅）（1年共12期）
美国国家地理幼儿版3-6岁（英文原版）National Geographic  Little Kids（1年共6期）（杂志订阅）
美国国家地理儿童版（英文原版）National Geographic Kids（1年共10期）（杂志订阅）
中国国家地理（1年共12期）（杂志订阅）
海洋探秘（1年共12期）（杂志订阅）
天府文化（1年共12期）（杂志订阅）
自然探秘(1年共12期）（杂志订阅）
户外探险（1年共6期）（杂志订阅）
旅游世界（1年共12期）（杂志订阅）
四川画报（1年共12期）（杂志订阅）
包邮 KiDS环球少年地理（与美国国家地理少儿版版权合作）（1年共12期）（杂志订阅）
包邮 中国国家地理（1年共12期）（杂志订阅）
动物大世界 中国儿童画报（1年共12期）（杂志订阅）【杂志铺专供】
快乐科学 快乐历史地理（1年共12期）（杂志订阅）
